# Pràctica amb ordinador 5. Proporcions de mestissatge.

## Objectiu
Aprendre com es fan i com s'interpreten els gràfics de **proporcions de mestissatge**, com el de la figura següent, treta de [Friedlaender et al. (2008)](https://doi.org/10.1371/journal.pgen.0040019). L'estudi s'encaminava a estudiar en detall l'estructura genètica de les poblacions humanes del Pacífic.

![](exemple.png)

<center><b>Figura 1</b>. Proporcions d'ancestralitat estimades en 1893 individus de 91 poblacions, a partir dels genotips en 687 microsatèl·lits i 203 insercions (Friedlaender et al., 2008).</center>

### Exercici 1. Revisem la interpretació intuïtiva d'aquesta figura:
1. Què indiquen els colors?
2. Què són les barres verticals?
3. Creus que existeix un valor de *K* més adequat que els altres?
4. Amb $K=2$, creus que és raonable la separació observada entre les poblacions oceàniques i pràcticament la resta del món?
5. Quin altre detall et crida la atenció?

## El model
Les figures com l'anterior assumeixen un model demogràfic concret, amb les assumpcions següents:
1. Les poblacions ancestrals van viure separades, van divergir per deriva genètica i poden haver-se mesclat recentment.
2. Les poblacions mestisses no han patit deriva genètica des del mestissatge: la seua constitució genètica s'explica adequadament per les proporcions de mestissatge i les freqüències al·lèliques inicials.
3. Totes les poblacions ancestrals estan ben representades en la mostra.

La figura següent, de Lawson et al. (2018), representa un exemple d'aquest model.

<center>
<img src="model.png" alt="Drawing" style="width: 200px;"/>
</center>
<center><b>Figura 2</b>. Model demogràfic de diferenciació i mestissatge entre 4 poblacions (a) i proporcions de mestissatge corresponents en una mostra d'individus de les 4 poblacions (Lawson et al. 2018).</center>

### Exercici 2. Quines poblacions humanes coneixes que podrien haver tingut una evolució semblant?

## Les dades
Les dades de partida són:

1. Una gran matriu **G**, dels genotips en *M* loci (normalment, milers d'SNPs) d'*N* individus;
2. i un número de poblacions ancestrals que suposem que van existir, *K*.

Friedlaender et al. (2008) van utilitzar dades genètiques de 890 loci en 1893 individus de 91 poblacions diferents. Nosaltres utilitzarem els genotips de 12 llops i 74 gossos de tres races, en prop de 77000 SNPs. Les dades estan a la carpeta `data`, en un format binari propi del programa `plink` (Chang et al. 2015). Per poder transformar-les a un format de text pla, cal tenir instal·lat el programa `plink`. Executant el bloc de codi següent s'instal·laran tres programes que farem servir durant la pràctica: `plink`, `admixture` i `evalAdmix`. Aquest bloc només cal executar-lo una vegada:

In [ ]:
system2('./installacio.sh')

Des d'aquest quadern Jupyter només podem executar ordres del llenguatge R. Per executar comandaments del sistema, els hem d'embolicar amb la funció `system2()` d'R.

El bloc següent generarà una versió llegible de les dades genètiques (la matriu **G**):

In [10]:
if (! file.exists('data/cf6.ped')) 
   system2('plink', args = c('--bfile', 'data/cf6',
                             '--out', 'data/cf6',
                             '--recode',
                             '--dog'),
          stdout = FALSE, stderr = FALSE)

### Exercici 3. Quina estructura té l'arxiu `data/cf6.ped`?

## Mètodes
El programa `admixture` utilitza la matriu **G** i el número de poblacions ancestrals *K* per *estimar* dues matrius: la matriu **P** de les freqüències al·lèliques dels *M* loci en cadascuna de les *K* poblacions ancestrals; i la matriu **Q** ($N\times K$), de les proporcions de loci que cada individu ha heretat de cada una de les *K* poblacions. El bloc de codi següent realitza l'estimació per a $K=2$. L'opció `--cv` instrueix el programa per estimar també una taxa d'error en l'assignació d'ancestralitats.

In [23]:
if (! file.exists('cf6.2.Q'))
    system2('admixture', args = c('--cv', 'data/cf6.bed', '2'),
           stdout = 'cf6.2.log')

### Exercici 4. En un altre bloc de codi a continuació escriu tu i executa les ordres per estimar les matrius **P** i **Q** amb valors de *K* entre 3 i 6.

## Resultats
És hora de visualitzar els resultats. Explora alguna de les matrius de proporcions de mestissatge, **Q**. Voràs que cada matriu té tantes files com individus i tantes columnes com poblacions ancestrals hem assumit. Les files sumen 1, perquè cada columna representa la proporció estimada del genoma que cada individu ha heretat de cada població ancestral. El codi següent representa cada fila de **Q** com una barra vertical amb fragments de colors diferents proporcionals a la fracció del genoma heretada de cada suposada població ancestral.

In [ ]:
Q2 <- matrix(scan('cf6.2.Q'), ncol = 2, byrow = TRUE)
options(repr.plot.width = 21, repr.plot.height = 7, repr.plot.res = 100)
barplot(t(Q2), col = c(2,3))

### Exercici 5. En blocs de codi addicionals, genera gràfics com l'anterior per a cada valor de *K*.

Un aspecte important és que el mètode utilitzat per `admixture` ignora per complet les *etiquetes* que podem haver assignat als individus. El coneixement previ sobre l'origen de les mostres pot utilitzar-se per interpretar els resultats. Els individus (és a dir, les files en la matriu **Q**) estan disposats en el mateix ordre en què aparixen els individus en la matriu **G** de genotips. La identitat dels individus podem trobar-la en les primeres columnes d'aquella matriu **G** (és a dir, a l'arxiu `data/cf6.ped`) o, per més comoditat, en l'arxiu `data/cf6.fam`. Obri aquest últim i observa les dues primeres columnes, que corresponen a una població assignada i un identificador únic d'individu. Les quatre poblacions a les quals pertanyen les nostres mostres són:

  |     | Població             | N  |
  | --- | -------------------- | -- |
  | GSH | Pastor alemany       | 12 |
  | SAW | Gos-llop Saarloos    | 18 |
  | CSW | Gos-llop txecoslovac | 44 |
  | WLF | Llop europeu         | 12 |

Podem afegir esta informació als gràfics anteriors amb les instruccions següents:

In [ ]:
metadata <- read.table('data/cf6.fam', col.names = c('pop', 'ind', 'dad', 'mom', 'sex', 'phe'),
                       colClasses = c('factor', 'character', 'NULL', 'NULL', 'NULL', 'NULL'))
row.names(Q2) <- metadata$ind
# "space=0" anula l'espai entre barres, fent que la posició de cada individu siga igual al seu número d'ordre.
# "las=2" fa que els noms dels individus s'escriguen perpendiculars a l'eix.
barplot(t(Q2), col = c(2, 3), space = 0, las = 2)
popPosition <- sapply(split(1:dim(metadata)[1], metadata$pop), mean)
popMaximum  <- sapply(split(1:dim(metadata)[1], metadata$pop), max)
axis(3, at = popPosition, labels = names(popPosition), tick = 0)
abline(v = popMaximum, lwd = 2, col = 'white')

### Exercici 6. Afegeix les etiquetes de població als gràfics amb valors de *K* majors i comenta els resultats.

## Avaluació dels resultats
El programa `admixture` ofereix un mètode per avaluar quin valor de *K* és més adequat per a les dades. Consisteix en identificar el valor de *K* amb un *error de validació creauada* menor. Aquests errors es troben al final dels arxius `.log`.

### Exercici 7. Ompli el vector `CVerrors` del bloc següent manualment amb els valors que trobaràs als arxius `.log`, i després executa el bloc per obtenir-ne una representació gràfica.

In [ ]:
CVerrors <- c(0, 0, 0, 0, 0)
K <- c(2, 3, 4, 5, 6)
options(repr.plot.width = 7, repr.plot.height = 7, repr.plot.res = 100)
plot(K, CVerrors, type = 'b')

### Bibliografia

- Chang et al. 2015. Second-generation PLINK: rising to the challenge of larger and richer datasets. *GigaScience* 4(1):s13742-015-0047-8. [https://doi.org/10.1186/s13742-015-0047-8](https://doi.org/10.1186/s13742-015-0047-8).
- Friedlaender et al. 2008. The genetic structure of Pacific islanders. *PLoS Genetics* 4(1):1-18. [https://doi.org/10.1371/journal.pgen.0040019](https://doi.org/10.1371/journal.pgen.0040019).
- Lawson, D.J., van Dorp, L. & Falush, D. 2018. A tutorial on how not to over-interpret STRUCTURE and ADMIXTURE bar plots. *Nat. Commun.* 9:3258. [https://doi.org/10.1038/s41467-018-05257-7](https://doi.org/10.1038/s41467-018-05257-7).